In [1]:
import random
import simpy

class VM:
    def __init__(self, id, cpu, ram):
        self.id = id
        self.cpu = cpu
        self.ram = ram
        self.cost = cpu / 10
        self.available_cpu = cpu
        self.available_ram = ram

class Task:
    def __init__(self, id, instructions, memory, deadline):
        self.id = id
        self.instructions = instructions
        self.memory = memory
        self.deadline = deadline

def simple_allocate(vms, task):
    for vm in vms:
        if vm.available_cpu >= task.instructions and vm.available_ram >= task.memory:
            return vm
    return None

def run_simulation(vms, tasks):
    env = simpy.Environment()
    
    def process_task(task):
        vm = simple_allocate(vms, task)
        if vm is None:
            return None
        
        vm.available_cpu -= task.instructions
        vm.available_ram -= task.memory
        
        execution_time = task.instructions / vm.cpu * 1000  # ms
        yield env.timeout(execution_time)
        
        vm.available_cpu += task.instructions
        vm.available_ram += task.memory
        
        cost = vm.cost * (execution_time / 1000)
        energy = vm.cpu * (execution_time / 1000) * 0.001
        
        return execution_time, cost, energy

    processes = [env.process(process_task(task)) for task in tasks]
    results = env.run(until=simpy.AllOf(env, processes))
    
    valid_results = [r.value for r in results if r.value is not None]
    if not valid_results:
        return None, None, None
    
    execution_times, costs, energies = zip(*valid_results)
    return max(execution_times), sum(costs), sum(energies)


In [2]:
# to prevent GA from printing output
import sys
import os
import contextlib

@contextlib.contextmanager
def suppress_output():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr


In [3]:
# GA
from deap import base, creator, tools, algorithms
import random
import numpy as np

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

def evaluate(individual, vms, tasks):
    vm_allocations = [vms[i] for i in individual]
    makespan, total_cost, total_energy = run_simulation(vm_allocations, tasks)
    return makespan, total_cost, total_energy

def ga_optimize(vms, tasks):
    toolbox = base.Toolbox()
    toolbox.register("attr_int", random.randint, 0, len(vms)-1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=len(tasks))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate, vms=vms, tasks=tasks)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(vms)-1, indpb=0.1)
    toolbox.register("select", tools.selTournament, tournsize=3)

    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)

    with suppress_output():
        algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=40, stats=stats, halloffame=hof)

    best = hof[0]
    return best.fitness.values

# ga_makespan, ga_cost, ga_energy = ga_optimize(vms, tasks)

In [4]:
# FWA
def fwa_optimize(vms,tasks):
    def generate_sparks(firework, n, amplitude, num_vms):
        return [
            [max(0, min(num_vms - 1, x + int(random.uniform(-amplitude, amplitude))))
             for x in firework]
            for _ in range(n)
        ]
    def fireworks_algorithm(vms, tasks, n_fireworks=5, n_sparks=30, n_iterations=40):
        num_vms = len(vms)
        num_tasks = len(tasks)

        # Initialize fireworks
        fireworks = [random.choices(range(num_vms), k=num_tasks) for _ in range(n_fireworks)]
        best_solution = None
        best_fitness = float('inf')

        for _ in range(n_iterations):
            all_solutions = fireworks.copy()

            # Generate sparks
            for firework in fireworks:
                fitness = sum(evaluate(firework, vms, tasks))
                if fitness < best_fitness:
                    best_fitness = fitness
                    best_solution = firework

                # Calculate amplitude, ensuring it's within a reasonable range
                if fitness == 0:
                    amplitude = 50  # Set a default amplitude if fitness is zero
                else:
                    amplitude = min(50, max(1, int(50 * best_fitness / fitness)))

                all_solutions.extend(generate_sparks(firework, n_sparks, amplitude, num_vms))

            # Evaluate and select best solutions
            all_solutions.sort(key=lambda x: sum(evaluate(x, vms, tasks)))
            fireworks = all_solutions[:n_fireworks]

        return best_solution, best_fitness

    best_solution, best_fitness = fireworks_algorithm(vms, tasks)
    return evaluate(best_solution, vms, tasks)

# fwa_makespan, fwa_cost, fwa_energy=fwa_optimize(vms,tasks)

In [5]:
# SQSA
def sqsa_optimize(vms, tasks, n_squirrels=30, n_iterations=40):
    squirrels = [random.choices(range(len(vms)), k=len(tasks)) for _ in range(n_squirrels)]
    fitness = [sum(evaluate(s, vms, tasks)) for s in squirrels]
    best_idx = fitness.index(min(fitness))
    
    for _ in range(n_iterations):
        for i in range(n_squirrels):
            if i != best_idx:
                for j in range(len(tasks)):
                    if random.random() < 0.5:
                        squirrels[i][j] = squirrels[best_idx][j]
                    else:
                        squirrels[i][j] = random.randint(0, len(vms)-1)
        
        fitness = [sum(evaluate(s, vms, tasks)) for s in squirrels]
        new_best_idx = fitness.index(min(fitness))
        if fitness[new_best_idx] < fitness[best_idx]:
            best_idx = new_best_idx

    return evaluate(squirrels[best_idx], vms, tasks)

# sqsa_makespan, sqsa_cost, sqsa_energy=sqsa_optimize(vms,tasks)

In [6]:
# BAT
def bat_optimize(vms, tasks, n_bats=30, n_iterations=40):
    bats = [random.choices(range(len(vms)), k=len(tasks)) for _ in range(n_bats)]
    velocities = [[0] * len(tasks) for _ in range(n_bats)]
    fitness = [sum(evaluate(b, vms, tasks)) for b in bats]
    best_bat = bats[fitness.index(min(fitness))]
    
    for _ in range(n_iterations):
        for i in range(n_bats):
            freq = random.random()
            for j in range(len(tasks)):
                velocities[i][j] += (bats[i][j] - best_bat[j]) * freq
                bats[i][j] = max(0, min(len(vms)-1, int(bats[i][j] + velocities[i][j])))
            
            if random.random() > 0.5:
                bats[i] = [max(0, min(len(vms)-1, best_bat[j] + int(random.gauss(0, 1))))
                           for j in range(len(tasks))]
        
        fitness = [sum(evaluate(b, vms, tasks)) for b in bats]
        best_idx = fitness.index(min(fitness))
        if fitness[best_idx] < sum(evaluate(best_bat, vms, tasks)):
            best_bat = bats[best_idx]

    return evaluate(best_bat, vms, tasks)

# bat_makespan, bat_cost, bat_energy=bat_optimize(vms,tasks)

In [7]:
# PSO
def pso_optimize(vms, tasks, n_particles=30, n_iterations=40):
    particles = [random.choices(range(len(vms)), k=len(tasks)) for _ in range(n_particles)]
    velocities = [[0] * len(tasks) for _ in range(n_particles)]
    personal_best = particles.copy()
    fitness = [sum(evaluate(p, vms, tasks)) for p in particles]
    global_best = particles[fitness.index(min(fitness))]
    
    for _ in range(n_iterations):
        for i in range(n_particles):
            for j in range(len(tasks)):
                r1, r2 = random.random(), random.random()
                velocities[i][j] = (0.5 * velocities[i][j] + 
                                    1 * r1 * (personal_best[i][j] - particles[i][j]) +
                                    2 * r2 * (global_best[j] - particles[i][j]))
                particles[i][j] = max(0, min(len(vms)-1, int(particles[i][j] + velocities[i][j])))
            
            current_fitness = sum(evaluate(particles[i], vms, tasks))
            if current_fitness < sum(evaluate(personal_best[i], vms, tasks)):
                personal_best[i] = particles[i]
                if current_fitness < sum(evaluate(global_best, vms, tasks)):
                    global_best = particles[i]

    return evaluate(global_best, vms, tasks)

# pso_makespan, pso_cost, pso_energy=pso_optimize(vms,tasks)

In [8]:
# BMO
def bmo_optimize(vms, tasks, n_barnacles=30, n_iterations=40):
    barnacles = [random.choices(range(len(vms)), k=len(tasks)) for _ in range(n_barnacles)]
    fitness = [sum(evaluate(b, vms, tasks)) for b in barnacles]
    best_barnacle = barnacles[fitness.index(min(fitness))]
    
    for _ in range(n_iterations):
        for i in range(n_barnacles):
            if random.random() < 0.5:  # Pseudo-fertilization
                partner = random.choice(barnacles)
                barnacles[i] = [b if random.random() < 0.5 else p 
                                for b, p in zip(barnacles[i], partner)]
            else:  # Larval development
                barnacles[i] = [max(0, min(len(vms)-1, x + int(random.gauss(0, 1))))
                                for x in barnacles[i]]
        
        fitness = [sum(evaluate(b, vms, tasks)) for b in barnacles]
        new_best = barnacles[fitness.index(min(fitness))]
        if sum(evaluate(new_best, vms, tasks)) < sum(evaluate(best_barnacle, vms, tasks)):
            best_barnacle = new_best

    return evaluate(best_barnacle, vms, tasks)

# bmo_makespan, bmo_cost, bmo_energy = bmo_optimize(vms, tasks)

In [9]:
# SSA
def ssa_optimize(vms, tasks, n_sparrows=30, n_iterations=40):
    sparrows = [random.choices(range(len(vms)), k=len(tasks)) for _ in range(n_sparrows)]
    fitness = [sum(evaluate(s, vms, tasks)) for s in sparrows]
    best_sparrow = sparrows[fitness.index(min(fitness))]
    
    for _ in range(n_iterations):
        for i in range(n_sparrows):
            if i < n_sparrows // 5:  # Discoverer
                sparrows[i] = [max(0, min(len(vms)-1, x + int(random.gauss(0, 1))))
                               for x in sparrows[i]]
            elif i < n_sparrows // 2:  # Joiner
                sparrows[i] = [best_sparrow[j] if random.random() < 0.5 else sparrows[i][j]
                               for j in range(len(tasks))]
            else:  # Scrounger
                sparrows[i] = [random.randint(0, len(vms)-1) if random.random() < 0.1 else sparrows[i][j]
                               for j in range(len(tasks))]
        
        fitness = [sum(evaluate(s, vms, tasks)) for s in sparrows]
        new_best = sparrows[fitness.index(min(fitness))]
        if sum(evaluate(new_best, vms, tasks)) < sum(evaluate(best_sparrow, vms, tasks)):
            best_sparrow = new_best

    return evaluate(best_sparrow, vms, tasks)

# ssa_makespan, ssa_cost, ssa_energy = ssa_optimize(vms, tasks)

In [10]:
# Setup
num_vms = 20
num_tasks = 100
vms = [VM(i, random.randint(1000, 3000), random.randint(1000, 2000)) for i in range(num_vms)]
tasks = [Task(i, random.randint(100, 500), random.randint(100, 500), random.randint(5, 50)) for i in range(num_tasks)]

In [11]:
makespan, total_cost, total_energy = run_simulation(vms, tasks)

In [12]:
ga_makespan, ga_cost, ga_energy = ga_optimize(vms, tasks)

In [13]:
fwa_makespan, fwa_cost, fwa_energy=fwa_optimize(vms,tasks)

In [14]:
sqsa_makespan, sqsa_cost, sqsa_energy=sqsa_optimize(vms,tasks)

In [15]:
bat_makespan, bat_cost, bat_energy=bat_optimize(vms,tasks)

In [16]:
pso_makespan, pso_cost, pso_energy=pso_optimize(vms,tasks)

In [17]:
bmo_makespan, bmo_cost, bmo_energy = bmo_optimize(vms, tasks)

In [18]:
ssa_makespan, ssa_cost, ssa_energy = ssa_optimize(vms, tasks)

In [19]:
def compare_results(results):
    for name, (makespan, cost, energy) in results.items():
        print(f"{name}:")
        print(f"  Makespan: {makespan}")
        print(f"  Cost: {cost}")
        print(f"  Energy: {energy}")
        print()

    best_makespan = min(results.items(), key=lambda x: x[1][0])
    best_cost = min(results.items(), key=lambda x: x[1][1])
    best_energy = min(results.items(), key=lambda x: x[1][2])

    print(f"Best Makespan: {best_makespan[0]}")
    print(f"Best Cost: {best_cost[0]}")
    print(f"Best Energy: {best_energy[0]}")
    
# Collect results
results = {
    "No Optimization": (makespan, total_cost, total_energy),
    "Genetic Algorithm": (ga_makespan, ga_cost, ga_energy),
    "Fireworks Algorithm": (fwa_makespan, fwa_cost, fwa_energy),
    "Squirrel Search Algorithm": (sqsa_makespan,sqsa_cost,sqsa_energy),
    "Bat Algorithm": (bat_makespan, bat_cost, bat_energy),
    "Particle Swarm Optimizer": (pso_makespan, pso_cost, pso_energy),
    "Barnacles Mating Optimizer": (bmo_makespan, bmo_cost, bmo_energy),
    "Sparrow Search Algorithm": (ssa_makespan, ssa_cost, ssa_energy)
}

compare_results(results)

No Optimization:
  Makespan: 303.32056194125164
  Cost: 2929.2000000000003
  Energy: 29.291999999999987

Genetic Algorithm:
  Makespan: 253.781512605042
  Cost: 2929.2000000000003
  Energy: 29.291999999999987

Fireworks Algorithm:
  Makespan: 303.20284697508896
  Cost: 765.8000000000001
  Energy: 7.658000000000001

Squirrel Search Algorithm:
  Makespan: 314.1153081510935
  Cost: 2470.8
  Energy: 24.70799999999999

Bat Algorithm:
  Makespan: 207.09745762711864
  Cost: 391.50000000000006
  Energy: 3.915

Particle Swarm Optimizer:
  Makespan: 292.7733168622607
  Cost: 2724.500000000001
  Energy: 27.244999999999987

Barnacles Mating Optimizer:
  Makespan: 352.3515805705474
  Cost: 2509.600000000001
  Energy: 25.095999999999993

Sparrow Search Algorithm:
  Makespan: 294.7118241235888
  Cost: 2508.1
  Energy: 25.080999999999992

Best Makespan: Bat Algorithm
Best Cost: Bat Algorithm
Best Energy: Bat Algorithm
